In [1]:
import pickle

from docuverse.engines.search_engine_config_params import DocUVerseConfig
from docuverse.utils import open_stream
from pymilvus import (
    MilvusClient,
    DataType,
    connections,
    utility,
    Collection
)
import numpy as np

from docuverse import SearchEngine
def load_cache(cache_file):
    return pickle.load(open_stream(cache_file))

# cache_file="/home/raduf/.local/share/elastic_ingestion/benchmark__beir_dev__quora____en__corpus.small.jsonl_512_100_True_all_gte-small.pickle.xz"
# cache_dense="/home/raduf/.local/share/elastic_ingestion/benchmark__beir_dev__quora__en__corpus.small.jsonl_512_100_True_all_slate.30m.english.rtrvr.pickle.xz"
# cache_hybrid="/home/raduf/.local/share/elastic_ingestion/benchmark__beir_dev__quora__en__corpus.small.jsonl_512_100_True_all_slate.30m.english.rtrvr.pickle.xz"
# ddata=load_cache(cache_dense)
# hdata=load_cache(cache_hybrid)
# data=ddata
# import json
# questions = [
#     "How can I get free gems in Clash of Clans?",
#     "How can I get free gems Clash of Clans?",
#     "How do you feel when someone upvotes your answer on Quora?",
#     "What are the best thriller movie in Hollywood?",
#     "What should someone do to overcome anxiety?"
# ]

In [2]:
# MODEL = ("/home/raduf/sandbox2/docuverse/models/slate.30m.english.rtrvr")
MODEL = ("ibm-granite/granite-embedding-30m-english")
data_list = []
from docuverse.utils.embeddings.dense_embedding_function import DenseEmbeddingFunction
model = DenseEmbeddingFunction(MODEL)

Running on the gpus:['NVIDIA GeForce RTX 4090', 'NVIDIA GeForce RTX 5090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090'], attention: sdpa
=== done initializing model


In [3]:
config=DocUVerseConfig("../experiments/sap/sap_milvus_dense.dev.granite30.rerank_IRL.100.yaml")
engine = SearchEngine(config)
output_file1="../output/debug1.json"
output_file2="../home-output/sap_milvus_dense.dev.granite30.rerank_IRL.100.retrieve.json"
output1 = engine.read_output(output_file1)
output2 = engine.read_output(output_file2)

OSError: models/reranker_125m_6Mar25 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [4]:
[len(o) for o in output1]

NameError: name 'output1' is not defined

In [51]:
scores1 = [np.array([o['score'] for o in out]) for out in output1]
scores2 = [np.array([o['score'] for o in out]) for out in output2]

In [59]:
scores1[6][:30]-scores2[6][:30]


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [60]:
first_non_zero_index = np.argmax(scores1[6][:30] - scores2[6][:30] != 0)
print(first_non_zero_index)

0


In [70]:
(scores1[8][:30] - scores2[8][:30])


array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -1.19417906e-03, -5.57303429e-05, -1.47163868e-03,
       -2.39670277e-03, -2.91305780e-03, -1.92660093e-03, -6.30736351e-04,
       -4.89830971e-04, -4.42147255e-04, -2.29024887e-03, -2.29084492e-03,
       -2.14707851e-03, -1.17588043e-03, -1.49905682e-03, -2.78359652e-03,
       -4.97066975e-03, -4.68391180e-03, -1.16276741e-03, -1.14387274e-03,
       -1.05226040e-03, -1.14589930e-03])

In [90]:
print(f"{output1[8][9].as_dict()}\n{output2[8][9].as_dict()}")

{'filestem': 'sap_152907', 'id': 'sap_152907.txt-0-1397', 'text': 'Spot Award Currency Program with Integration to Employee Central | SAP Help Portal\n\n Spot Award Currency Program with Integration to Employee Central \n\n\n\nThe detailed procedure for testing the scope item  Spot Award Currency Program with Integration to Employee Central after solution activation is described in the following. Each process step is covered in its own section.\n\nReward and Recognition are events that take place at times other than during the regular compensation planning period. They are often subject to the same budget, eligibility, guidelines, and approval process as focal events, but since participation is limited, they do not lend themselves well to worksheet tools. As a result, effectively managing Reward and Recognition and compensation changes is a major challenge for a compensation team.\n\nNote\n\nThe Reimagined Home Page (Latest Home Page) will be pushed to all customer instances that are n

In [100]:
item1=output1[8][9]
item2=output2[8][9]
question=output1[8].question
text1=item1.get_text()
text2=item2.get_text()
qtext=question.text

In [123]:
v=engine.retriever.encode_data([text1, text2], batch_size=1)
q=engine.retriever.encode_data([qtext], batch_size=1)[0]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [114]:
def cosine(A, B):
    # return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
    return np.dot(A, B)

In [142]:
print(f"cos1 of {output1[8][9]['id']}={cosine(v[0], q)}, cos2 of {output2[8][9]['id']}={cosine(v[1], q)}")

cos1 of sap_152907.txt-0-1397=0.7266868214712701, cos2 of sap_049304.txt-0-436=0.7278808457713997


In [129]:
item2.id

'sap_049304.txt-0-436'

In [137]:
client = MilvusClient("http://localhost:19530")
collection_name="sap_milvus_dense_granite30m_512_100_20250318"
# client.list_indexes(collection_name)
collection = Collection(collection_name)

ConnectionNotExistException: <ConnectionNotExistException: (code=1, message=should create connection first.)>

In [145]:
res1=client.query(collection_name=collection_name,
                  filter=f"id=='{output1[8][9]['id']}'")[0]
res2=client.query(collection_name=collection_name,
                  filter=f"id=='{output2[8][9]['id']}'")[0]

In [147]:
v1=res1['embeddings']
v2=res2['embeddings']

In [149]:
print(f"cos1 of {output1[8][9]['id']}={cosine(v1, q)}, cos2 of {output2[8][9]['id']}={cosine(v2, q)}")

cos1 of sap_152907.txt-0-1397=0.7266868024869264, cos2 of sap_049304.txt-0-436=0.7278808947285881


In [153]:
client.search

[0.013371208682656288,
 0.019740650430321693,
 0.10312164574861526,
 0.049145665019750595,
 0.004826135467737913,
 -0.009538259357213974,
 -0.00010587887663859874,
 -0.0077717723324894905,
 0.017958907410502434,
 -0.007306836079806089,
 -0.0021458156406879425,
 -0.006775213405489922,
 -0.03426913544535637,
 -0.038776692003011703,
 -0.004530788864940405,
 0.02548758126795292,
 -0.03603638708591461,
 0.015997733920812607,
 -0.014054141007363796,
 0.021336521953344345,
 -0.00673261983320117,
 -0.036887288093566895,
 -0.008027433417737484,
 -0.02847108617424965,
 -0.029421715065836906,
 -0.1023915559053421,
 -0.01523595955222845,
 -0.023716557770967484,
 0.016527066007256508,
 -0.17107081413269043,
 0.025537315756082535,
 -0.008401384577155113,
 -0.004546136595308781,
 -0.0012872060760855675,
 -0.07602319121360779,
 -0.07594925165176392,
 -0.009037421084940434,
 0.0019562228117138147,
 0.03318873047828674,
 -0.011110550723969936,
 -0.02482442557811737,
 0.04475919529795647,
 0.019561259075

In [ ]:
search_params = {"metric_type": "IP", "params": {"nprobe": 180000, "efSearch": 180000}}
sres=client.search(collection_name=collection_name, search_params=search_params, data=[q], limit=100, output_fields=['text', '_id', 'id', 'filestem'])[0]

In [211]:
coll_name="sap_milvus_dense_granite30m_512_100_20250329b_FLAT_MNone"
search_params = {"metric_type": "IP", "params": {}}
sres_flat=client.search(collection_name=coll_name, search_params=search_params, data=[q], limit=100, output_fields=['text', '_id', 'id', 'filestem'])[0]

In [212]:
def filter(res):
    seen = {}
    out = []
    for r in res:
        if r['entity']['filestem'] not in seen:
           out.append(r)
           seen[r['entity']['filestem']] = True
    return out

In [213]:
fres=filter(sres)[:10]

In [214]:
[f"{f['entity']['filestem']}: {f['distance']}" for f in fres]

['sap_152815: 0.8090283274650574',
 'sap_087423: 0.7593492865562439',
 'sap_153069: 0.7465554475784302',
 'sap_152668: 0.7455179691314697',
 'sap_174667: 0.7429876327514648',
 'sap_152801: 0.7414252161979675',
 'sap_152912: 0.7354713678359985',
 'sap_153111: 0.7341508269309998',
 'sap_152787: 0.7338778972625732',
 'sap_152907: 0.7266868352890015']

In [196]:
[f"{a['filestem']}: {a['score']}" for a in output1[8][:10]]

['sap_152815: 0.8090283870697021',
 'sap_087423: 0.7593492269515991',
 'sap_153069: 0.7465554475784302',
 'sap_152668: 0.745517909526825',
 'sap_174667: 0.7429876327514648',
 'sap_152801: 0.7414251565933228',
 'sap_152912: 0.7354713082313538',
 'sap_153111: 0.734150767326355',
 'sap_152787: 0.733877956867218',
 'sap_152907: 0.7266867756843567']

In [197]:
[f"{a['filestem']}: {a['score']}" for a in output2[8][:10]]

['sap_152815: 0.8090283870697021',
 'sap_087423: 0.7593492269515991',
 'sap_153069: 0.7465554475784302',
 'sap_152668: 0.745517909526825',
 'sap_174667: 0.7429876327514648',
 'sap_152801: 0.7414251565933228',
 'sap_152912: 0.7354713082313538',
 'sap_153111: 0.734150767326355',
 'sap_152787: 0.733877956867218',
 'sap_049304: 0.7278809547424316']

In [198]:
res2

{'filestem': 'sap_049304',
 'embeddings': [np.float32(-0.012917706),
  np.float32(0.018926173),
  np.float32(0.015240759),
  np.float32(0.06878987),
  np.float32(0.008769169),
  np.float32(0.0137228025),
  np.float32(-0.025523411),
  np.float32(-0.015647216),
  np.float32(0.071482636),
  np.float32(-0.010946404),
  np.float32(-0.019661002),
  np.float32(-0.017748933),
  np.float32(0.0046190834),
  np.float32(-0.01130594),
  np.float32(0.06786583),
  np.float32(-0.022437626),
  np.float32(-0.00023853216),
  np.float32(-0.01051921),
  np.float32(-0.035050366),
  np.float32(0.011295003),
  np.float32(-0.027482245),
  np.float32(-0.07758613),
  np.float32(0.00050947553),
  np.float32(-0.027472451),
  np.float32(-0.03632765),
  np.float32(-0.05273766),
  np.float32(-0.036828123),
  np.float32(-0.091831565),
  np.float32(0.0032490096),
  np.float32(-0.07737107),
  np.float32(0.018103126),
  np.float32(-0.011909966),
  np.float32(-0.06686616),
  np.float32(-0.069085054),
  np.float32(-0.08269